In [63]:

import pandas as pd
import psycopg2
from sqlalchemy import create_engine, inspect, insert
from sqlalchemy.orm import sessionmaker
from sqlalchemy.sql import text
import datetime
import os
from tqdm import tqdm

import sys
from importlib import reload
sys.path.append('../../') 
sys.path.append('../src/')
from credentials import DB_username, DB_password, DB_host, DB_name

In [2]:
today = datetime.date.today()
print("Today's date:", today)

Today's date: 2024-02-23


__connect to DB__



In [4]:
# create engine 
for_engine = f'postgresql+psycopg2://{DB_username}:{DB_password}@{DB_host}/{DB_name}'
engine = create_engine(for_engine)

# control checking
inspector = inspect(engine)
display(inspector.get_schema_names())

# make query
query = text('select * from data.currency')
with engine.connect() as conn:
    result = conn.execute(query)
conn.close()
display(result.fetchall())


['data', 'information_schema', 'layer_0', 'layer_1']

[(8, 'BTCUSD', datetime.date(2024, 1, 31))]

__pandas features__

In [53]:
# SELECTs
display(pd.read_sql_query('SELECT * FROM data.currency limit 3;', con=engine))
display(pd.read_sql(text("SELECT * FROM data.currency where creation_date=:var1"), con=engine, params={"var1": "2024-02-01"}))

#INSERTS pandas.dataframe
df = pd.DataFrame({'currency':['var1', 'var2'], 'creation_date':['2024-01-01', '2024-01-02']})
# ATTENTION!!! create table with name='' if it not exist 
df.to_sql(name='currency', schema='data', con=engine, if_exists='append', index=False)

,id,currency,creation_date
0,8,BTCUSD,2024-01-31
1,9,BTCUSD,2024-02-01
2,10,BTCUSD,2024-02-01


,id,currency,creation_date
0,9,BTCUSD,2024-02-01
1,10,BTCUSD,2024-02-01


2

__add to table 'currency' row__

In [7]:
# make insert
query = text('''INSERT INTO data.currency (currency, creation_date) VALUES ('BTCUSD', :today)''')
with engine.connect() as conn:
    conn.execute(query, {'today':str(today)})
    conn.commit()
conn.close()

___


## Filling the DB table 'klines'

In [75]:
# Get the list of all files and directories
path = "D://myGit//BTC_HFT_pet//data//raw//new"
dir_list = os.listdir(path)

path_file = path + '//' + dir_list[0]
data = pd.read_csv(path_file, header=0, compression='gzip')

In [76]:
# create engine 
for_engine = f'postgresql+psycopg2://{DB_username}:{DB_password}@{DB_host}/{DB_name}'
engine = create_engine(for_engine)

In [77]:

for i in tqdm(dir_list):
    path_file = path + '//' + i
    data = pd.read_csv(path_file, header=0, compression='gzip')
    data.columns=['datetime', 'open', 'high', 'low', 'close', 'volume']
    data['datetime'] = pd.to_datetime(data['datetime'])
    data['category'] = 4
    data['interval'] = 6
    data['symbol'] = 1
    data['source'] = 1
    data['datetime_created'] = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    data.to_sql(name='klines', schema='layer_0', con=engine, if_exists='append', index=False)

100%|██████████| 47/47 [00:02<00:00, 18.44it/s]


___